In [1]:
import sys
import os
import cv2
import numpy as np
import pandas as pd
import json
from datetime import datetime
from tensorflow.keras.models import load_model

from PyQt5.QtWidgets import (
    QMainWindow, QApplication, QWidget, QVBoxLayout, QLabel, QProgressBar,
    QTextEdit, QMenuBar, QMenu, QAction, QFileDialog, QMessageBox, QDialog,
    QPushButton, QTableWidget, QTableWidgetItem, QHeaderView, QHBoxLayout,
    QTabWidget, QSlider, QSplitter, QGroupBox
)
from PyQt5.QtCore import Qt, QThread, pyqtSignal, pyqtSlot, QTimer
from PyQt5.QtGui import QPixmap, QImage, QIcon, QFont, QPainter, QBrush, QPen, QColor

# ==================== ПРОАКТИВНЫЙ ПРОГНОЗИРУЮЩИЙ МОДЕЛЬ ====================

class ProactivePredictor:
    def __init__(self, base_model):
        self.base_model = base_model
        self.prediction_horizon = 8
        self.confidence_threshold = 0.6
        
    def predict_escalation(self, current_sequence):
        """Прогнозирование эскалации на основе текущей последовательности"""
        if len(current_sequence) < 16:
            return 0.0, "Not enough data"
            
        # Анализ тренда движения
        motion_trend = self._analyze_motion_trend(current_sequence)
        intensity_trend = self._analyze_intensity_trend(current_sequence)
        
        # Прогноз вероятности насилия
        current_pred = self.base_model.predict(np.expand_dims(current_sequence, axis=0))[0]
        current_violence_prob = float(current_pred[1])
        
        # Прогнозируем развитие на основе трендов
        predicted_prob = min(1.0, current_violence_prob + motion_trend * 0.3 + intensity_trend * 0.2)
        
        if predicted_prob > self.confidence_threshold and current_violence_prob < 0.3:
            return predicted_prob, "ESCALATION LIKELY"
        elif predicted_prob > self.confidence_threshold:
            return predicted_prob, "CONFLICT CONTINUES"
        else:
            return predicted_prob, "STABLE SITUATION"

    def _analyze_motion_trend(self, sequence):
        """Анализ тренда движения между кадрами"""
        if len(sequence) < 3:
            return 0.0
            
        motions = []
        for i in range(1, len(sequence)):
            motion = np.mean(np.abs(sequence[i] - sequence[i-1]))
            motions.append(motion)
        
        if len(motions) >= 3:
            recent_trend = np.mean(motions[-3:]) - np.mean(motions[-6:-3])
            return max(-1.0, min(1.0, recent_trend * 10))
        return 0.0

    def _analyze_intensity_trend(self, sequence):
        """Анализ тренда интенсивности изменений"""
        intensities = [np.std(frame) for frame in sequence]
        if len(intensities) >= 4:
            recent_avg = np.mean(intensities[-2:])
            previous_avg = np.mean(intensities[-4:-2])
            return (recent_avg - previous_avg) / max(previous_avg, 0.001)
        return 0.0

# ==================== ВИЗУАЛИЗАЦИЯ В РЕАЛЬНОМ ВРЕМЕНИ ====================

class RealTimeComparisonWidget(QWidget):
    def __init__(self):
        super().__init__()
        self.reactive_results = []
        self.proactive_results = []
        self.setMinimumHeight(200)
        self.setMaximumHeight(250)
        
    def add_detection(self, reactive_conf, proactive_conf, proactive_alert, time):
        self.reactive_results.append({
            'time': time, 'confidence': reactive_conf, 'type': 'reactive'
        })
        self.proactive_results.append({
            'time': time, 'confidence': proactive_conf, 'alert': proactive_alert, 'type': 'proactive'
        })
        
        if len(self.reactive_results) > 50:
            self.reactive_results.pop(0)
            self.proactive_results.pop(0)
            
        self.update()
        
    def paintEvent(self, event):
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)
        
        # Фон
        painter.fillRect(0, 0, self.width(), self.height(), QColor(250, 250, 250))
        
        # Заголовок
        painter.setFont(QFont("Arial", 10, QFont.Bold))
        painter.drawText(0, 10, "REAL-TIME COMPARISON:")
        
        # Легенда
        painter.setFont(QFont("Arial", 9))
        painter.setPen(QColor(200, 50, 50))
        painter.drawText(200, 20, "Red - Reactive")
        painter.setPen(QColor(50, 100, 200))
        painter.drawText(350, 20, "Blue - Proactive")
        
        if not self.reactive_results:
            return
            
        # Графики уверенности
        max_time = max(max(r['time'] for r in self.reactive_results), 10)
        max_conf = max(max(r['confidence'] for r in self.reactive_results + self.proactive_results), 0.8)
        
        # Реактивный график (красный)
        painter.setPen(QPen(QColor(200, 50, 50), 2))
        for i in range(1, len(self.reactive_results)):
            x1 = self.time_to_x(self.reactive_results[i-1]['time'], max_time)
            y1 = self.conf_to_y(self.reactive_results[i-1]['confidence'], max_conf)
            x2 = self.time_to_x(self.reactive_results[i]['time'], max_time)
            y2 = self.conf_to_y(self.reactive_results[i]['confidence'], max_conf)
            painter.drawLine(x1, y1, x2, y2)
            
        # Проактивный график (синий)
        painter.setPen(QPen(QColor(50, 100, 200), 2))
        for i in range(1, len(self.proactive_results)):
            x1 = self.time_to_x(self.proactive_results[i-1]['time'], max_time)
            y1 = self.conf_to_y(self.proactive_results[i-1]['confidence'], max_conf)
            x2 = self.time_to_x(self.proactive_results[i]['time'], max_time)
            y2 = self.conf_to_y(self.proactive_results[i]['confidence'], max_conf)
            painter.drawLine(x1, y1, x2, y2)
            
            # Маркеры предупреждений
            if self.proactive_results[i]['alert'] != "STABLE SITUATION":
                painter.setBrush(QBrush(QColor(255, 255, 0)))
                painter.drawEllipse(x2-4, y2-4, 8, 8)
                
        # Текущие значения
        if self.reactive_results:
            last_reactive = self.reactive_results[-1]
            last_proactive = self.proactive_results[-1]
            
            painter.setFont(QFont("Arial", 10, QFont.Bold))
            
            # Реактивное значение
            painter.setPen(QColor(200, 50, 50))
            reactive_text = f"Reactive: {last_reactive['confidence']:.1%}"
            painter.drawText(10, 180, reactive_text)
            
            # Проактивное значение
            painter.setPen(QColor(50, 100, 200))
            proactive_text = f"Proactive: {last_proactive['confidence']:.1%}"
            painter.drawText(10, 200, proactive_text)
            
            # Статус предупреждения
            if last_proactive['alert'] != "STABLE SITUATION":
                painter.setPen(QColor(200, 100, 0))
                alert_text = f"ALERT: {last_proactive['alert']}"
                painter.drawText(200, 190, alert_text)
                
                # Подсветка преимущества
                if last_proactive['confidence'] > last_reactive['confidence']:
                    advantage = last_proactive['confidence'] - last_reactive['confidence']
                    painter.setPen(QColor(0, 150, 0))
                    painter.drawText(400, 190, f"ADVANTAGE: +{advantage:.1%}")

    def time_to_x(self, time, max_time):
        return 50 + (time / max_time) * (self.width() - 100)
        
    def conf_to_y(self, confidence, max_conf):
        return 150 - (confidence / max_conf) * 100

class LiveDetectionOverlay:
    def __init__(self):
        self.overlay_text = ""
        self.alert_level = 0
        self.proactive_advantage = 0.0
        
    def draw_on_frame(self, frame):
        """Рисует overlay поверх видео с фиксированным размером"""
        # Сохраняем оригинальный размер
        original_h, original_w = frame.shape[:2]
        
        # Создаем копию для overlay
        overlay = frame.copy()
        
        # Полупрозрачный фон для текста
        overlay[50:150, 10:400] = overlay[50:150, 10:400] * 0.7
        
        # Цвет в зависимости от уровня тревоги
        if self.alert_level == 2:
            color = (0, 0, 255)  # Красный
            alert_text = "DANGER!"
        elif self.alert_level == 1:
            color = (0, 165, 255)  # Оранжевый
            alert_text = "WARNING"
        else:
            color = (0, 255, 0)  # Зеленый
            alert_text = "NORMAL"
            
        # Текст overlay на английском
        cv2.putText(overlay, f"ViolenceTracker Live", (20, 70), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.putText(overlay, f"Status: {alert_text}", (20, 100), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        cv2.putText(overlay, f"Proactive advantage: {self.proactive_advantage:.1%}", (20, 130), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
                   
        if self.overlay_text:
            cv2.putText(overlay, self.overlay_text, (20, 160), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
                       
        return overlay

# ==================== МОДИФИЦИРОВАННЫЙ ПРОЦЕССОР ВИДЕО ====================

class AdvancedVideoProcessor:
    update_frame = None
    update_result = None
    
    def __init__(self, model, image_size=(64, 64), sequence_length=16):
        self.model = model
        self.proactive_predictor = ProactivePredictor(model)
        self.image_size = image_size
        self.sequence_length = sequence_length
        self.processing = True
        self.overlay = LiveDetectionOverlay()
        
        # Статистика для сравнения
        self.comparison_stats = {
            'reactive_detections': 0,
            'proactive_alerts': 0,
            'time_advantages': [],
            'false_positives_reduction': 0
        }

    def analyze_video_advanced(self, video_path, emit_frame=False, emit_result=False):
        """Продвинутый анализ с одновременным реактивным и проактивным детектированием"""
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            return self._create_error_result(video_path, "error")

        fps = cap.get(cv2.CAP_PROP_FPS) or 30
        frame_sequence = []
        combined_results = []
        
        frame_count = 0
        while cap.isOpened() and self.processing:
            ret, frame = cap.read()
            if not ret:
                break

            # Сохраняем оригинальный размер кадра
            original_frame = frame.copy()
            
            # Обработка кадра для модели
            processed_frame = self._preprocess_frame(frame)
            frame_sequence.append(processed_frame)
            
            if len(frame_sequence) > self.sequence_length:
                frame_sequence.pop(0)
                
            current_time = frame_count / fps
            
            if len(frame_sequence) == self.sequence_length:
                # РЕАКТИВНОЕ обнаружение
                reactive_confidence = self._reactive_detection(frame_sequence)
                
                # ПРОАКТИВНОЕ прогнозирование
                proactive_confidence, proactive_alert = self.proactive_predictor.predict_escalation(frame_sequence)
                
                # Сравнение и анализ преимущества
                advantage = max(0, proactive_confidence - reactive_confidence)
                detection_comparison = self._compare_detections(
                    reactive_confidence, proactive_confidence, proactive_alert, current_time
                )
                
                combined_results.append(detection_comparison)
                
                # Обновление overlay
                self._update_overlay(reactive_confidence, proactive_confidence, proactive_alert, advantage)
            
            # Emit кадр с overlay (оригинальный размер)
            if emit_frame:
                frame_with_overlay = self.overlay.draw_on_frame(original_frame)
                self.update_frame.emit(frame_with_overlay)
                
            frame_count += 1

        cap.release()
        return self._compile_final_results(combined_results, video_path)

    def _reactive_detection(self, frame_sequence):
        """Реактивное обнаружение насилия"""
        input_data = np.expand_dims(frame_sequence, axis=0)
        prediction = self.model.predict(input_data, verbose=0)[0]
        return float(prediction[1])

    def _compare_detections(self, reactive_conf, proactive_conf, proactive_alert, current_time):
        """Сравнение результатов двух методов"""
        
        advantage = proactive_conf - reactive_conf
        advantage_seconds = advantage * 5
        
        # Определение типа события
        if proactive_alert != "STABLE SITUATION" and reactive_conf < 0.3:
            event_type = "PROACTIVE WARNING"
            self.comparison_stats['proactive_alerts'] += 1
            if advantage_seconds > 0:
                self.comparison_stats['time_advantages'].append(advantage_seconds)
        elif reactive_conf > 0.7:
            event_type = "REACTIVE DETECTION" 
            self.comparison_stats['reactive_detections'] += 1
        else:
            event_type = "NORMAL SITUATION"
            
        message = (f"Time: {current_time:.1f}s | "
                  f"Reactive: {reactive_conf:.1%} | "
                  f"Proactive: {proactive_conf:.1%} | "
                  f"Advantage: {advantage:+.1%}")
                  
        if proactive_alert != "STABLE SITUATION":
            message += f" | {proactive_alert}"
            
        return {
            'time': current_time,
            'reactive_confidence': reactive_conf,
            'proactive_confidence': proactive_conf,
            'proactive_alert': proactive_alert,
            'advantage': advantage,
            'event_type': event_type,
            'message': message,
            'timestamps': [current_time]
        }

    def _update_overlay(self, reactive_conf, proactive_conf, alert, advantage):
        """Обновление overlay для отображения на видео"""
        self.overlay.proactive_advantage = advantage
        
        if alert != "STABLE SITUATION" and proactive_conf > reactive_conf:
            self.overlay.alert_level = 2 if proactive_conf > 0.7 else 1
            self.overlay.overlay_text = f"Proactive alert: {alert}"
        else:
            self.overlay.alert_level = 0
            self.overlay.overlay_text = ""

    def _preprocess_frame(self, frame):
        """Предобработка кадра для модели (не меняет оригинальный кадр)"""
        resized = cv2.resize(frame, self.image_size)
        return resized / 255.0

    def _create_error_result(self, video_path, status):
        return {
            "filename": os.path.basename(video_path),
            "status": status,
            "total_frames": 0,
            "violence_frames": 0,
            "max_confidence": 0.0
        }

    def _compile_final_results(self, combined_results, video_path):
        """Компиляция итоговых результатов"""
        if not combined_results:
            return self._create_error_result(video_path, "no_data")
            
        reactive_confs = [r['reactive_confidence'] for r in combined_results]
        proactive_confs = [r['proactive_confidence'] for r in combined_results]
        
        return {
            "filename": os.path.basename(video_path),
            "total_frames": len(combined_results),
            "max_reactive_confidence": max(reactive_confs),
            "max_proactive_confidence": max(proactive_confs),
            "avg_advantage": np.mean([r['advantage'] for r in combined_results]),
            "proactive_alerts": self.comparison_stats['proactive_alerts'],
            "reactive_detections": self.comparison_stats['reactive_detections'],
            "time_advantages": self.comparison_stats['time_advantages'],
            "combined_results": combined_results
        }

# ==================== ОБНОВЛЕННЫЙ ПОТОК ОБРАБОТКИ ====================

class AdvancedAnalysisThread(QThread):
    update_progress = pyqtSignal(int, int, str)
    update_result = pyqtSignal(str, list, dict)
    update_frame = pyqtSignal(np.ndarray)
    update_comparison = pyqtSignal(float, float, str, float)
    video_file_processed = pyqtSignal()

    def __init__(self, video_path, model, image_size, sequence_length):
        super().__init__()
        self.video_path = video_path
        self.processor = AdvancedVideoProcessor(model, image_size, sequence_length)
        self.processor.update_frame = self.update_frame
        self.processor.update_result = self.update_result

    def run(self):
        video_files = [f for f in os.listdir(self.video_path)
                      if f.lower().endswith(('.mp4', '.avi', '.mov'))]
        total_videos = len(video_files)
        
        for i, video_file in enumerate(video_files):
            if not self.processor.processing:
                break
                
            self.update_progress.emit(i+1, total_videos, f"ADVANCED ANALYSIS: {video_file}")
            full_path = os.path.join(self.video_path, video_file)
            
            # Запуск продвинутого анализа
            result = self.processor.analyze_video_advanced(
                full_path, 
                emit_frame=True, 
                emit_result=True
            )
            
            # Emit данные для графика сравнения
            for comp_result in result.get('combined_results', []):
                self.update_comparison.emit(
                    comp_result['reactive_confidence'],
                    comp_result['proactive_confidence'], 
                    comp_result['proactive_alert'],
                    comp_result['time']
                )
                
            self.video_file_processed.emit()

# ==================== ОБНОВЛЕННЫЙ ГЛАВНЫЙ ИНТЕРФЕЙС ====================

class AdvancedViolenceDetectorApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.is_fullscreen = False
        self.image_size = (64, 64)
        self.SEQUENCE_LENGTH = 16
        
        self.video_path = None
        self.current_file_data = {}
        self.results = []
        self.simplified_mode = False
        
        # Фиксированный размер для видео
        self.video_width = 640
        self.video_height = 480
        
        self.setup_ui()
        
        try:
            self.model = load_model('best_model_Unidirectional_LSTM.h5')
        except Exception as e:
            QMessageBox.critical(self, "Ошибка", f"Не удалось загрузить модель:\n{str(e)}")
            sys.exit(1)

    def setup_ui(self):
        self.setWindowTitle("ViolenceTracker Pro - Reactive + Proactive Analysis")
        self.setGeometry(100, 100, 1400, 900)
        self.setStyleSheet(self.get_styles())

        # Главный layout
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        layout = QHBoxLayout(central_widget)
        layout.setContentsMargins(5, 5, 5, 5)

        # Левая панель - видео и управление
        left_panel = QWidget()
        left_panel.setMaximumWidth(800)
        left_layout = QVBoxLayout(left_panel)

        # Видео панель с фиксированным размером
        self.video_label = QLabel()
        self.video_label.setAlignment(Qt.AlignCenter)
        self.video_label.setFixedSize(self.video_width, self.video_height)
        self.video_label.setStyleSheet("""
            background-color: black; 
            color: white; 
            border: 2px solid #ccc;
            qproperty-alignment: AlignCenter;
        """)
        self.video_label.setText("Video will appear here...")
        left_layout.addWidget(self.video_label)

        # Прогресс бар
        self.progress_bar = QProgressBar()
        left_layout.addWidget(self.progress_bar)

        # Консоль
        self.console = QTextEdit()
        self.console.setReadOnly(True)
        self.console.setMaximumHeight(150)
        left_layout.addWidget(self.console)

        # Правая панель - визуализации
        right_panel = QWidget()
        right_layout = QVBoxLayout(right_panel)

        # График сравнения в реальном времени
        comparison_group = QGroupBox("REAL-TIME METHODS COMPARISON")
        comparison_layout = QVBoxLayout(comparison_group)
        self.comparison_widget = RealTimeComparisonWidget()
        comparison_layout.addWidget(self.comparison_widget)
        right_layout.addWidget(comparison_group)

        # Статистика преимуществ
        stats_group = QGroupBox("ADVANTAGE STATISTICS")
        stats_layout = QVBoxLayout(stats_group)
        self.stats_label = QLabel("Start analysis to see statistics...")
        self.stats_label.setStyleSheet("font-size: 11px; color: #666; padding: 10px;")
        stats_layout.addWidget(self.stats_label)
        right_layout.addWidget(stats_group)

        # Информация о текущем анализе
        info_group = QGroupBox("CURRENT ANALYSIS INFO")
        info_layout = QVBoxLayout(info_group)
        self.info_label = QLabel(
            "Proactive method analyzes:\n"
            "• Motion trends\n• Intensity changes\n• Escalation patterns\n\n"
            "Reactive method:\n• Current frame state"
        )
        self.info_label.setStyleSheet("font-size: 11px; padding: 10px;")
        info_layout.addWidget(self.info_label)
        right_layout.addWidget(info_group)

        right_layout.addStretch()

        layout.addWidget(left_panel)
        layout.addWidget(right_panel)

        self.create_menu()

    def create_menu(self):
        menubar = self.menuBar()

        recognition_menu = menubar.addMenu("Analysis")
        
        advanced_action = QAction("Advanced Analysis (Both Methods)", self)
        advanced_action.triggered.connect(self.select_folder_advanced)
        recognition_menu.addAction(advanced_action)
        
        reactive_action = QAction("Reactive Analysis Only", self)
        reactive_action.triggered.connect(self.select_folder_frame_by_frame)
        recognition_menu.addAction(reactive_action)

        demo_menu = menubar.addMenu("Demo")
        live_demo_action = QAction("Live Advantage Demo", self)
        live_demo_action.triggered.connect(self.run_live_demo)
        demo_menu.addAction(live_demo_action)

    def select_folder_advanced(self):
        """Запуск продвинутого анализа с обоими методами"""
        self.console.clear()
        self.results = []
        self.comparison_widget.reactive_results = []
        self.comparison_widget.proactive_results = []
        
        folder = QFileDialog.getExistingDirectory(self, "Select folder with videos for advanced analysis")
        if folder:
            self.video_path = folder
            self.console.append("=== STARTING ADVANCED ANALYSIS ===")
            self.console.append("Reactive + Proactive methods working in parallel")
            self.console.append("Blue graph - Proactive, Red - Reactive\n")
            self.process_videos_advanced()

    def process_videos_advanced(self):
        """Обработка видео продвинутым методом"""
        if not self.video_path:
            return
            
        self.thread = AdvancedAnalysisThread(
            self.video_path, self.model, (64, 64), 16
        )
        
        # Подключаем все сигналы
        self.thread.update_progress.connect(self.update_progress)
        self.thread.update_frame.connect(self.update_video_frame)
        self.thread.update_result.connect(self.handle_advanced_result)
        self.thread.update_comparison.connect(self.update_comparison_display)
        self.thread.video_file_processed.connect(self.on_video_processed)
        
        self.thread.start()

    @pyqtSlot(float, float, str, float)
    def update_comparison_display(self, reactive_conf, proactive_conf, alert, time):
        """Обновление графика сравнения в реальном времени"""
        self.comparison_widget.add_detection(reactive_conf, proactive_conf, alert, time)
        
        # Обновление статистики
        advantage = proactive_conf - reactive_conf
        if advantage > 0.1 and alert != "STABLE SITUATION":
            stats_text = (f"PROACTIVE ADVANTAGE: +{advantage:.1%}\n"
                         f"Alert: {alert}\n"
                         f"Time: {time:.1f}s\n"
                         f"Reactive: {reactive_conf:.1%} | Proactive: {proactive_conf:.1%}")
            self.stats_label.setText(stats_text)
            self.stats_label.setStyleSheet("color: green; font-weight: bold; font-size: 11px;")

    @pyqtSlot(str, list, dict)
    def handle_advanced_result(self, text, timestamps, result):
        """Обработка результатов продвинутого анализа"""
        self.console.append(text)
        
        # Выделяем важные предупреждения
        if "PROACTIVE WARNING" in text:
            self.console.append("🚨 PROACTIVE WARNING DETECTED!")
            
        self.results.append(result)

    @pyqtSlot(np.ndarray)
    def update_video_frame(self, frame):
        """Обновление видео с overlay и фиксированным размером"""
        try:
            # Конвертация для PyQt
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            h, w, ch = frame_rgb.shape
            
            # Создаем QImage
            q_img = QImage(frame_rgb.data, w, h, ch * w, QImage.Format_RGB888)
            
            # Масштабируем до фиксированного размера
            pixmap = QPixmap.fromImage(q_img).scaled(
                self.video_width,
                self.video_height,
                Qt.KeepAspectRatio,
                Qt.SmoothTransformation
            )
            
            self.video_label.setPixmap(pixmap)
            
        except Exception as e:
            print(f"Error updating frame: {e}")

    @pyqtSlot(int, int, str)
    def update_progress(self, current, total, message):
        self.progress_bar.setValue(int((current / total) * 100))
        self.console.append(f"[{datetime.now().strftime('%H:%M:%S')}] {message}")

    def on_video_processed(self):
        """Завершение обработки видео"""
        self.console.append("\n✅ Video analysis completed")
        self.show_final_comparison()

    def show_final_comparison(self):
        """Показать итоговое сравнение методов"""
        if not self.results:
            return
            
        # Анализ преимуществ проактивного метода
        all_advantages = []
        proactive_alerts = 0
        reactive_detections = 0
        
        for result in self.results:
            for comp_result in result.get('combined_results', []):
                advantage = comp_result['advantage']
                if advantage > 0:
                    all_advantages.append(advantage)
                    
                if comp_result['event_type'] == "PROACTIVE WARNING":
                    proactive_alerts += 1
                elif comp_result['event_type'] == "REACTIVE DETECTION":
                    reactive_detections += 1
        
        if all_advantages:
            avg_advantage = np.mean(all_advantages)
            max_advantage = max(all_advantages)
            
            summary = (f"\n=== ANALYSIS SUMMARY ===\n"
                      f"Proactive warnings: {proactive_alerts}\n"
                      f"Reactive detections: {reactive_detections}\n"
                      f"Average advantage: {avg_advantage:.1%}\n"
                      f"Maximum advantage: {max_advantage:.1%}\n"
                      f"Proactive method warned {avg_advantage*5:.1f} seconds earlier!")
            
            self.console.append(summary)
            self.stats_label.setText(summary)

    def run_live_demo(self):
        """Запуск живой демонстрации"""
        self.console.clear()
        self.console.append("=== LIVE ADVANTAGE DEMONSTRATION ===")
        self.console.append("Load video to start demonstration...")
        self.select_folder_advanced()

    # Старые методы для обратной совместимости
    def select_folder_frame_by_frame(self):
        self.simplified_mode = False
        self.console.clear()
        self.results = []
        
        folder = QFileDialog.getExistingDirectory(self, "Select folder with videos")
        if folder:
            self.video_path = folder
            self.console.append("Mode: Reactive analysis only")

    def get_styles(self):
        return """
            QMainWindow {
                background-color: #F5F5F5;
            }
            QGroupBox {
                font-weight: bold;
                border: 2px solid #CCCCCC;
                border-radius: 5px;
                margin-top: 1ex;
                padding-top: 10px;
            }
            QGroupBox::title {
                subcontrol-origin: margin;
                left: 10px;
                padding: 0 5px 0 5px;
                color: #2C3E50;
            }
            QProgressBar {
                border: 1px solid #CCCCCC;
                border-radius: 5px;
                height: 20px;
                text-align: center;
                background: #FFFFFF;
            }
            QProgressBar::chunk {
                background-color: #3498DB;
                border-radius: 5px;
            }
            QTextEdit {
                background: #FFFFFF;
                border: 1px solid #CCCCCC;
                border-radius: 5px;
                font: 12px 'Consolas';
                color: #333333;
            }
        """

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = AdvancedViolenceDetectorApp()
    window.show()
    sys.exit(app.exec_())

e:\miniconda\envs\violence_env\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


1/1 [==============================] - 0s 40ms/step


C:\Users\Loca\AppData\Local\Temp\ipykernel_3872\351100726.py:130: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  painter.drawLine(x1, y1, x2, y2)
C:\Users\Loca\AppData\Local\Temp\ipykernel_3872\351100726.py:139: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  painter.drawLine(x1, y1, x2, y2)


1/1 [==============================] - 0s 40ms/step


C:\Users\Loca\AppData\Local\Temp\ipykernel_3872\351100726.py:144: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  painter.drawEllipse(x2-4, y2-4, 8, 8)


1/1 [==============================] - 0s 47ms/step


SystemExit: 0

e:\miniconda\envs\violence_env\lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
